In [1]:
import os
import zipfile
from PIL import Image
import csv
import numpy as np
import pandas as pd
from keras.models import load_model
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import warnings 
warnings.filterwarnings("ignore")

In [3]:
# unzipping the dataset of images
with zipfile.ZipFile('./ML_TESTING.zip', 'r') as zip_ref:
        zip_ref.extractall('./')

In [4]:
imgs = [img for img in os.listdir('./ML_TESTING') if img.endswith('.ppm')]
for img in imgs:
    img_name = img
    path = './ML_TESTING/' + img_name
    img = Image.open(path)
    # converting image to grayscale
    img = img.convert("L")
    # resizing image to 28x28
    img = img.resize((28,28))
    # saving image as png
    name = './external_images/' + img_name.split('.ppm')[0] + '.png'
    img.save(name)

In [5]:
header = ['img_path', 'shape_pred', 'type_pred']
csv_data = []
imgs = [img for img in os.listdir('./external_images') if img.endswith('.png')]
for img in imgs:
    path = './external_images/' + img
    row = [path]
    csv_data.append(row)

In [6]:
#writing collected data to the csv file
with open('./external_data.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        # write the header
        writer.writerow(header)
        # write multiple rows
        writer.writerows(csv_data)

In [7]:
df = pd.read_csv('./external_data.csv')

In [8]:
df.shape

(104, 3)

In [9]:
df.head()

,img_path,shape_pred,type_pred
0,./external_images/00132.png,NaN,NaN
1,./external_images/00085.png,NaN,NaN
2,./external_images/00052.png,NaN,NaN
3,./external_images/00251.png,NaN,NaN
4,./external_images/00053.png,NaN,NaN


In [10]:
dt = pd.read_csv('./external_images_actual.csv')

In [11]:
dt.head(7)

,img_id,sign_shape,sign_type
0,00094.png,0,0
1,00226.png,0,0
2,00225.png,0,0
3,00010.png,0,0
4,00007.png,0,0
5,00251.png,1,1
6,00132.png,1,1


In [12]:
dt['sign_shape'] = dt['sign_shape'].astype('str')
dt['sign_type'] = dt['sign_type'].astype('str')

In [13]:
df['img_path'][0]

'./external_images/00132.png'

In [14]:
name = df['img_path'][0].split('/')[2]
name

'00132.png'

In [15]:
name = '00019.png'
s = dt[dt['img_id'] == name]
print(s['sign_shape'])
# s = s['sign_shape'].iloc[0]
# print(int(s))

60    4
Name: sign_shape, dtype: object


In [16]:
@tf.autograph.experimental.do_not_convert
def make_predictions(model, to_predict):
    predictions = []
    actual = []
    for i in range(df.shape[0]):
        # loading the grayscale images
        img_path = df['img_path'][i]
        img = image.load_img(img_path, target_size=(28, 28), color_mode="grayscale")
        # creating a batch of one image each
        img_array = image.img_to_array(img)
        img_batch = np.expand_dims(img_array, axis=0)
        # normalising pixel values
        img_preprocessed = img_batch/255.0
        # predicting
        prediction = model.predict(img_preprocessed, verbose=0)
        predicted_class = np.argmax(prediction)
        predictions.append(predicted_class)
        if (to_predict == "shape"):
            df['shape_pred'] = predicted_class
            img_id = df['img_path'][i].split('/')[2]
            actual_shape = dt[dt['img_id'] == img_id]
#             actual_shape = int(actual_shape)
            actual_shape = actual_shape['sign_shape'].iloc[0]
            actual.append(int(actual_shape))
        else:
            df['type_pred'] = predicted_class
            img_id = df['img_path'][i].split('/')[2]
            actual_type = dt[dt['img_id'] == img_id]
#             actual_type = int(actual_type)
            actual_type = actual_type['sign_type'].iloc[0]
            actual.append(int(actual_type))
    return predictions, actual

In [17]:
@tf.autograph.experimental.do_not_convert
def print_confusion_matrix(predicted, actual, to_predict):
    print(classification_report(actual, predicted))
    if (to_predict == "shape"):
        print(confusion_matrix(actual, predicted))
    else:
        labels = [0,1,4,5,6,7,8,9,10,12,13,15]
        print(confusion_matrix(actual, predicted, labels=labels))

In [18]:
# Supressing all the warnings for the purpose of documentation
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [19]:
# loading the saved model
model = load_model("./model_lenet.h5")
model_lenet_predictions, actual = make_predictions(model, "shape")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [20]:
model_lenet_predictions[0].dtype

dtype('int64')

In [21]:
print_confusion_matrix(model_lenet_predictions, actual, "shape")

              precision    recall  f1-score   support

           0       0.33      0.40      0.36         5
           1       0.50      0.71      0.59         7
           2       0.00      0.00      0.00         3
           3       0.86      0.86      0.86        37
           4       0.78      0.67      0.72        52

    accuracy                           0.71       104
   macro avg       0.50      0.53      0.51       104
weighted avg       0.75      0.71      0.73       104

[[ 2  0  0  0  3]
 [ 1  5  0  0  1]
 [ 1  0  0  0  2]
 [ 1  0  0 32  4]
 [ 1  5  6  5 35]]


In [22]:
# loading the saved model
model = load_model("./model_lenet2.h5")
model_lenet2_predictions, actual = make_predictions(model, "shape")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [23]:
print_confusion_matrix(model_lenet2_predictions, actual, "shape")

              precision    recall  f1-score   support

           0       0.80      0.80      0.80         5
           1       1.00      0.43      0.60         7
           2       0.17      0.33      0.22         3
           3       0.72      0.78      0.75        37
           4       0.76      0.73      0.75        52

    accuracy                           0.72       104
   macro avg       0.69      0.62      0.62       104
weighted avg       0.75      0.72      0.73       104

[[ 4  0  0  0  1]
 [ 0  3  0  0  4]
 [ 0  0  1  0  2]
 [ 1  0  2 29  5]
 [ 0  0  3 11 38]]


In [24]:
# loading the saved model
model = load_model("./model_vgg.h5")
model_vgg_predictions, actual = make_predictions(model, "shape")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [25]:
print_confusion_matrix(model_vgg_predictions, actual, "shape")

              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       1.00      0.71      0.83         7
           2       0.33      0.33      0.33         3
           3       0.95      0.97      0.96        37
           4       0.92      0.92      0.92        52

    accuracy                           0.91       104
   macro avg       0.81      0.79      0.79       104
weighted avg       0.92      0.91      0.91       104

[[ 5  0  0  0  0]
 [ 0  5  0  0  2]
 [ 0  0  1  0  2]
 [ 1  0  0 36  0]
 [ 0  0  2  2 48]]


In [26]:
# loading the saved model
model = load_model("./new_model_vgg2.h5")
new_model_vgg2_predictions, actual = make_predictions(model, "type")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [27]:
print_confusion_matrix(new_model_vgg2_predictions, actual, "type")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      0.71      0.83         7
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00        11
           7       0.00      0.00      0.00        26
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00        13
          12       0.00      0.00      0.00         7
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00        12

    accuracy                           0.10       104
   macro avg       0.13      0.11      0.12       104
weighted avg       0.12   

In [28]:
# loading the saved model
model = load_model("./new_model_vgg.h5")
new_model_vgg_predictions, actual = make_predictions(model, "type")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [29]:
print_confusion_matrix(new_model_vgg_predictions, actual, "type")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      0.71      0.83         7
           2       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00        11
           7       0.00      0.00      0.00        26
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00        13
          12       0.00      0.00      0.00         7
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00        12

    accuracy                           0.10       104
   macro avg       0.14      0.12      0.13       104
weighted avg       0.12      0.10      0.10       104

[[ 5  0  0  0  0  0  0  0

In [30]:
# loading the saved model
model = load_model("./new_model_lenet.h5")
new_model_lenet_predictions, actual = make_predictions(model, "type")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [31]:
print_confusion_matrix(new_model_lenet_predictions, actual, "type")

              precision    recall  f1-score   support

           0       0.67      0.80      0.73         5
           1       0.75      0.43      0.55         7
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00        11
           7       0.00      0.00      0.00        26
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00        13
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         7
          13       0.04      1.00      0.07         1
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00        12

    accuracy                           0.08       104
   macro avg       0.09   

In [32]:
# loading the saved model
model = load_model("./new_model_lenet2.h5")
new_model_lenet2_predictions, actual = make_predictions(model, "type")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [33]:
print_confusion_matrix(new_model_lenet2_predictions, actual, "type")

              precision    recall  f1-score   support

           0       0.67      0.80      0.73         5
           1       0.86      0.86      0.86         7
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00        11
           7       0.09      0.04      0.05        26
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00        13
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         7
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00        12

    accuracy                           0.11       104
   macro avg       0.10   